In [1]:
import pandas as pd
import numpy as np
import os
import requests

#LENDO INFO MANUAL

In [2]:
df_acfi = pd.read_excel("../input/pat.xlsx", sheet_name="AçoesFII")
df_ac_int = pd.read_excel("../input/pat.xlsx", sheet_name="Stocks")
df_banco = pd.read_excel("../input/pat.xlsx", sheet_name="Bancos")

# LENDO EXTRATOS NUBANK

In [3]:
# Specify the directory path
directory_path = "../input/Extrato Conta"

# Initialize an empty DataFrame to store the combined data
df_extrato = pd.DataFrame()

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Concatenate the current DataFrame with the combined DataFrame
        df_extrato = pd.concat([df_extrato, df], ignore_index=True)

df_extrato["Data"] = pd.to_datetime(df_extrato["Data"], format="%d/%m/%Y")

df_extrato = df_extrato.sort_values("Data")
# Display the combined DataFrame
display(df_extrato.head())

,Data,Valor,Identificador,Descrição
337,2018-07-30,500.00,5b5eeab8-72e8-46de-94c4-147b4574391e,Transferência Recebida - Marcos Vinicius Polle...
53,2018-08-03,1800.00,5b642f0c-ad50-40e4-a561-8d71abde49b1,Transferência Recebida - Marcos Vinicius Polle...
54,2018-08-06,-256.50,5b68976c-7e0e-456c-b43b-c000bb82b118,Pagamento de boleto efetuado - Banco do Brasil...
55,2018-08-06,76.50,5b68ac0a-ff64-4d55-a31c-a34150b822d4,Transferência Recebida - Thadeu Luiz Barbosa D...
772,2018-09-05,1042.44,5b8fae07-bd36-4f15-bf20-94d383d80ee4,Transferência Recebida - Marcos Vinicius Polle...


In [4]:
df_extrato_consolidado = df_extrato.copy()

In [5]:
# df_extrato_consolidado["tipo"] = (
#     df_extrato_consolidado["Descrição"].str.split("-").str.get(0)
# )
df_extrato_consolidado["tipo"] = (
    df_extrato_consolidado["Descrição"].str.split("-").str.get(0) +
    " - " +
    df_extrato_consolidado["Descrição"].str.split("-").str.get(1)
)

df_extrato_consolidado["title"] = df_extrato_consolidado["tipo"].fillna(df_extrato["Descrição"])

df_extrato_consolidado["pessoa"] = (
    df_extrato_consolidado["Descrição"].str.split("-").str.get(1)
)

df_extrato_consolidado.to_csv(
    "../data/consolidado/df_extrato_consolidado.csv", index=False, sep=","
)

In [6]:
# Specify the directory path
directory_path = "../input/rendimento nubank"

# Initialize an empty DataFrame to store the combined data
df_rendimento = pd.DataFrame()

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Concatenate the current DataFrame with the combined DataFrame
        df_rendimento = pd.concat([df_rendimento, df], ignore_index=True)

# df_rendimento["Data"] = pd.to_datetime(df_rendimento["Data"],  format="%d-%m-%Y")

df_rendimento = df_rendimento.sort_values("Data")
# Display the combined DataFrame
display(df_rendimento.head())

,Data,Entradas,Saídas,IR,IOF,Rendimentos,Saldo Líquido
0,2018-07-29,0.0,0.0,0.0,0.0,0.00,0.00
1,2018-07-30,500.0,0.0,0.0,0.0,0.00,500.00
2,2018-07-31,0.0,0.0,0.0,0.0,0.12,500.00
3,2018-08-01,0.0,0.0,0.0,0.0,0.12,500.01
4,2018-08-02,0.0,0.0,0.0,0.0,0.12,500.03


In [7]:
df_rendimento.to_csv("../data/consolidado/df_rendimento.csv", index=False, sep=",")

# Faturas nubank

In [8]:
# Specify the directory path
directory_path = "../input/Faturas/NU"

# Initialize an empty DataFrame to store the combined data
df_faturas_nu = pd.DataFrame()

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Concatenate the current DataFrame with the combined DataFrame
        df_faturas_nu = pd.concat([df_faturas_nu, df], ignore_index=True)

df_faturas_nu["date"] = pd.to_datetime(
    df_faturas_nu["date"], format="%Y-%m-%d", errors="coerce"
)


df_faturas_nu = df_faturas_nu.sort_values("date").reset_index(drop=True)
df_faturas_nu["amount"] = pd.to_numeric(df_faturas_nu["amount"], errors="coerce")

# Display the combined DataFrame
display(df_faturas_nu.head())
display(df_faturas_nu.shape)

,date,category,title,amount
0,2017-12-06,serviços,Recarga Tim,20.00
1,2017-12-11,transporte,Uber do Brasil Tecnolo,18.53
2,2017-12-11,outros,Mercpago,37.91
3,2017-12-11,restaurante,Comuna,32.00
4,2017-12-11,serviços,Vortice Consultoria,37.90


(2760, 4)

In [9]:
# date	Estabelecimento	Portador	amount	Parcela	title	category
df_faturas_nu["Estabelecimento"] = df_faturas_nu["title"]

df_faturas_nu["IF"] = "NU"
df_faturas_nu["Portador"] = "MARCOS POLLERY"
extracted_title = (
    df_faturas_nu["Estabelecimento"].str.extract(r"\*(.*?)\s*(\d+/\d+|$)").get(0)
)
df_faturas_nu["title"] = extracted_title.fillna(df_faturas_nu["Estabelecimento"])

df_faturas_nu["Parcela"] = (
    df_faturas_nu["Estabelecimento"]
    .str.extract(r"(\d+/\d+)$")
    .replace("/", " de ", regex=True)
)

df_faturas_nu

,date,category,title,amount,Estabelecimento,IF,Portador,Parcela
0,2017-12-06,serviços,Recarga Tim,20.00,Recarga Tim,NU,MARCOS POLLERY,NaN
1,2017-12-11,transporte,Uber do Brasil Tecnolo,18.53,Uber do Brasil Tecnolo,NU,MARCOS POLLERY,NaN
2,2017-12-11,outros,Mercpago,37.91,Mercpago,NU,MARCOS POLLERY,NaN
3,2017-12-11,restaurante,Comuna,32.00,Comuna,NU,MARCOS POLLERY,NaN
4,2017-12-11,serviços,Vortice Consultoria,37.90,Vortice Consultoria,NU,MARCOS POLLERY,NaN
...,...,...,...,...,...,...,...,...
2755,2023-12-27,restaurante,App Grupo M,67.58,Madero 164*App Grupo M,NU,MARCOS POLLERY,NaN
2756,2023-12-28,reversal_upfront_national_settled,"App Grupo""",-76.70,"Estorno de ""Jeronimo Nov*App Grupo""",NU,MARCOS POLLERY,NaN
2757,2023-12-28,restaurante,Cabana Burger S.A.,62.91,Ifd*Cabana Burger S.A.,NU,MARCOS POLLERY,NaN
2758,2023-12-28,lazer,Steam,60.01,Pag*Steam,NU,MARCOS POLLERY,NaN


In [10]:
df_faturas_nu.loc[~df_faturas_nu["Parcela"].isna()]

,date,category,title,amount,Estabelecimento,IF,Portador,Parcela
16,2018-01-04,outros,Blizzard Ente,30.80,Payu*Blizzard Ente 2/2,NU,MARCOS POLLERY,2 de 2
38,2018-03-04,casa,Ortobom 2/2,41.00,Ortobom 2/2,NU,MARCOS POLLERY,2 de 2
59,2018-05-04,eletrônicos,Super Celular 2/2,44.95,Super Celular 2/2,NU,MARCOS POLLERY,2 de 2
141,2018-10-04,casa,Tok Stok 2/12,66.58,Tok Stok 2/12,NU,MARCOS POLLERY,2 de 12
186,2018-11-04,casa,Tok Stok 3/12,66.58,Tok Stok 3/12,NU,MARCOS POLLERY,3 de 12
...,...,...,...,...,...,...,...,...
2711,2023-11-28,serviços,Nuvem Wasportseues,38.43,Pg *Nuvem Wasportseues 2/6,NU,MARCOS POLLERY,2 de 6
2712,2023-11-28,lazer,Uniao Crossfit 6/6,216.00,Uniao Crossfit 6/6,NU,MARCOS POLLERY,6 de 6
2713,2023-11-28,lazer,Steam,17.71,Pag*Steam 3/3,NU,MARCOS POLLERY,3 de 3
2714,2023-11-28,lazer,Steam,76.63,Pag*Steam 2/3,NU,MARCOS POLLERY,2 de 3


In [11]:
df_faturas_nu.loc[df_faturas_nu["amount"] < 0].head()

,date,category,title,amount,Estabelecimento,IF,Portador,Parcela
15,2018-01-02,payment,Pagamento recebido,-374.30,Pagamento recebido,NU,MARCOS POLLERY,NaN
31,2018-02-07,payment,Pagamento recebido,-541.18,Pagamento recebido,NU,MARCOS POLLERY,NaN
39,2018-03-09,payment,Pagamento recebido,-212.62,Pagamento recebido,NU,MARCOS POLLERY,NaN
46,2018-04-09,payment,Pagamento recebido,-115.00,Pagamento recebido,NU,MARCOS POLLERY,NaN
60,2018-05-04,payment,Pagamento recebido,-378.77,Pagamento recebido,NU,MARCOS POLLERY,NaN


In [12]:
df_faturas_nu.to_csv("../data/consolidado/df_faturas_nu.csv", index=False, sep=",")

# Faturas xp

In [13]:
# Specify the directory path
directory_path = "../input/Faturas/XP"

# Initialize an empty DataFrame to store the combined data
df_faturas_xp = pd.DataFrame()

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, sep=";")

        # Concatenate the current DataFrame with the combined DataFrame
        df_faturas_xp = pd.concat([df_faturas_xp, df], ignore_index=True)

df_faturas_xp["Data"] = pd.to_datetime(
    df_faturas_xp["Data"], format="%d/%m/%Y", errors="coerce"
)


df_faturas_xp = df_faturas_xp.sort_values("Data").reset_index(drop=True)
df_faturas_xp["Valor"] = (
    pd.to_numeric(
        df_faturas_xp["Valor"].replace("[^\\d.]", "", regex=True).str.replace(",", "."),
        errors="coerce",
    )
    / 100
)


# Display the combined DataFrame
display(df_faturas_xp.tail(30))
display(df_faturas_xp.shape)

,Data,Estabelecimento,Portador,Valor,Parcela
460,2023-11-27,MP*TOPBLEND,MARCOS POLLERY,30.71,2 de 7
461,2023-11-30,Pagamentos Validos Normais,MARCOS POLLERY,702.56,-
462,2023-12-01,UBER* TRIP,MARCOS POLLERY,16.46,-
463,2023-12-01,PIZZARIA DOSJOSEFA LTD,MARCOS POLLERY,50.49,-
464,2023-12-01,UBER* PENDING,MARCOS POLLERY,14.99,-
465,2023-12-03,UBER* PENDING,MARCOS POLLERY,19.99,-
466,2023-12-03,FILIAL 21 NORTE SHOPPI,MARCOS POLLERY,55.00,-
467,2023-12-06,NESTLE BRASIL LTDA.,MARCOS POLLERY,9.68,2 de 10
468,2023-12-06,NESTLE BRASIL LTDA.,MARCOS POLLERY,9.73,1 de 10
469,2023-12-06,MP *MELIMAIS,MARCOS POLLERY,17.99,-


(490, 5)

In [14]:
df_faturas_xp["title"] = df_faturas_xp["Estabelecimento"].str.split("*").str.get(-1)

# date	category	title	amount

df_faturas_xp = df_faturas_xp.rename({"Data": "date", "Valor": "amount"}, axis=1)

df_faturas_xp["category"] = np.nan

df_faturas_xp["IF"] = "XP"

df_faturas_xp["Parcela"] = df_faturas_xp["Parcela"].replace("-", np.nan)
df_faturas_xp

,date,Estabelecimento,Portador,amount,Parcela,title,category,IF
0,2022-09-27,IFD*EX-TOURO - BURGER VIL,MARCOS POLLERY,37.00,NaN,EX-TOURO - BURGER VIL,NaN,XP
1,2022-09-29,PG *WOLLNER,MARCOS POLLERY,54.72,4 de 4,WOLLNER,NaN,XP
2,2022-09-29,PG *WOLLNER,MARCOS POLLERY,54.72,2 de 4,WOLLNER,NaN,XP
3,2022-09-29,IFD*MCDONALDS - GUANABARA,MARCOS POLLERY,60.79,NaN,MCDONALDS - GUANABARA,NaN,XP
4,2022-09-29,PG *WOLLNER,MARCOS POLLERY,54.70,1 de 4,WOLLNER,NaN,XP
...,...,...,...,...,...,...,...,...
485,2024-01-08,EC *MELIMAIS,MARCOS POLLERY,17.99,NaN,MELIMAIS,NaN,XP
486,2024-01-11,IFD*GARRINHA PIZZARIA VIS,MARCOS POLLERY,29.00,NaN,GARRINHA PIZZARIA VIS,NaN,XP
487,2024-01-12,EC*ALIPAYBRASIL,MARCOS POLLERY,345.71,NaN,ALIPAYBRASIL,NaN,XP
488,2024-01-14,NESTLE BRASIL LTDA.,MARCOS POLLERY,25.62,1 de 10,NESTLE BRASIL LTDA.,NaN,XP


In [15]:
df_faturas_xp.to_csv("../data/consolidado/df_faturas_xp.csv", index=False, sep=",")

#UNINDO FATURAS

In [16]:
# tirando as negativas de pagamento de fatura
df_faturas = pd.concat(
    [
        df_faturas_nu.loc[
            ~df_faturas_nu["Estabelecimento"].str.contains("Pagamento recebido")
        ],
        df_faturas_xp,
    ]
).sort_values("date")

In [17]:
# deixando parcela xp igual parcela nubank
df_faturas["date2"] = df_faturas["date"]
# Filter rows where "IF" is equal to "XP"
xp_rows = df_faturas[df_faturas["IF"] == "XP"]

# Extract the first item from the split lists and convert to numeric
first_items = pd.to_numeric(xp_rows["Parcela"].str.split("de").str[0], errors="coerce")

# Update the "date" column based on the condition
condition = first_items > 1
for index, value in first_items[condition].items():
    df_faturas.at[index, "date"] += pd.DateOffset(months=int(value) - 1)

df_faturas.sort_values("date")

df_faturas.to_csv("../data/consolidado/df_faturas.csv", index=False, sep=",")

In [18]:
df_faturas[(df_faturas["IF"] == "XP") & ~(df_faturas["Parcela"].isna())]

,date,category,title,amount,Estabelecimento,IF,Portador,Parcela,date2
4,2022-09-29,NaN,WOLLNER,54.70,PG *WOLLNER,XP,MARCOS POLLERY,1 de 4,2022-09-29
5,2022-11-29,NaN,WOLLNER,54.72,PG *WOLLNER,XP,MARCOS POLLERY,3 de 4,2022-09-29
2,2022-10-29,NaN,WOLLNER,54.72,PG *WOLLNER,XP,MARCOS POLLERY,2 de 4,2022-09-29
1,2022-12-29,NaN,WOLLNER,54.72,PG *WOLLNER,XP,MARCOS POLLERY,4 de 4,2022-09-29
8,2023-01-30,NaN,CAFES,8.22,PARC=105MERCADOPAGO*CAFES,XP,MARCOS POLLERY,5 de 5,2022-09-30
...,...,...,...,...,...,...,...,...,...
471,2023-12-12,NaN,RENNER,34.01,RENNER,XP,MARCOS POLLERY,1 de 5,2023-12-12
478,2024-01-15,NaN,GRINDING,8.86,PAG*GRINDING,XP,MARCOS POLLERY,2 de 3,2023-12-15
477,2023-12-15,NaN,GRINDING,8.88,PAG*GRINDING,XP,MARCOS POLLERY,1 de 3,2023-12-15
481,2023-12-20,NaN,FARFETCHBR,210.25,FARFETCHBR,XP,MARCOS POLLERY,1 de 4,2023-12-20


# downloads

In [6]:
import os
import requests

import sys 
sys.path.append('G:\\Projetinhos\\Patrimonio')  
from load_data.password import *




# Iterate through each folder path
for folder_path in folder_paths:
    # Define the GitHub API endpoint for the current folder path
    api_url = f"https://api.github.com/repos/{github_username}/{github_repo}/contents/{folder_path}"

    # Make a GET request to the GitHub API to retrieve the contents of the folder
    response = requests.get(api_url, headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the response JSON
        files = response.json()

        # Specify the local directory where you want to save the downloaded files
        local_directory = f"../data/downloaded/{folder_path.replace('/', '_')}"

        # Create the local directory if it doesn't exist
        os.makedirs(local_directory, exist_ok=True)

        # Download only files that end with "_historical_data.csv"
        for file_info in files:
            # Get the file name
            file_name = os.path.basename(file_info["name"])

            # Check if the file name ends with "_historical_data.csv"
            if file_name.endswith("_historical_data.csv"):
                # Define the URL for downloading the raw content of the file
                raw_url = file_info["download_url"]

                # Make a GET request to download the file
                file_response = requests.get(raw_url, headers=headers)

                # Check if the request was successful (status code 200)
                if file_response.status_code == 200:
                    # Save the content to a local file
                    local_path = os.path.join(local_directory, file_name)
                    with open(local_path, "wb") as local_file:
                        local_file.write(file_response.content)
                    print(f"Downloaded: {file_name}")
                else:
                    print(f"Failed to download file: {file_name}")
    else:
        print(
            f"Failed to retrieve folder contents for {folder_path}. Status code: {response.status_code}"
        )

Downloaded: currencies_historical_data.csv
Downloaded: inflacao_historical_data.csv
Downloaded: selic_historical_data.csv
Downloaded: ABT_historical_data.csv
Downloaded: ARZZ3.SA_historical_data.csv
Downloaded: BCFF11.SA_historical_data.csv
Downloaded: BMOB3.SA_historical_data.csv
Downloaded: BOVA11.SA_historical_data.csv
Downloaded: BPAN4.SA_historical_data.csv
Downloaded: BRCR11.SA_historical_data.csv
Downloaded: BREW11.SA_historical_data.csv
Downloaded: BRK-B_historical_data.csv
Downloaded: CAML3.SA_historical_data.csv
Downloaded: CASH3.SA_historical_data.csv
Downloaded: CGRA4.SA_historical_data.csv
Downloaded: CYRE3.SA_historical_data.csv
Downloaded: DIVO11.SA_historical_data.csv
Downloaded: EIMI.L_historical_data.csv
Downloaded: FLRY3.SA_historical_data.csv
Downloaded: FSLR_historical_data.csv
Downloaded: GGRC11.SA_historical_data.csv
Downloaded: HAPV3.SA_historical_data.csv
Downloaded: HCTR11.SA_historical_data.csv
Downloaded: HGLG11.SA_historical_data.csv
Downloaded: INBR32.SA_h